In [1]:
import xarray as xr
import numpy as np
from utils import (
     detrend, monthly_anomalies, corr_with_ttest,reg_slope,
)

In [2]:
eq = dict(lat=0,lon=slice(140,260))
tpart = slice('1958-01','2018-12')

## Data preparation 

In [3]:
t_grid_file = xr.open_zarr('data/ORCA025.L46-KFS006_TroPA.zarr')

### Nemo SSH 

In [4]:
ssh_a = monthly_anomalies(t_grid_file.sossheig.sel(**eq,time=tpart)).compute()

In [5]:
ssh_y = detrend(ssh_a.compute())

In [6]:
def season(x):
    res=np.zeros((12,61,481))
    for i in range(0,12):
        for j in range(0,61):
            res[i,j,:] = x.where((x['time.month'] == i+1) & (x['time.year'] == j+1958), drop=True).drop_vars('time')

    season_x = xr.DataArray(
        data=res,
        dims=["month", "year", "lon"],
        coords=dict(
            month=x.where(x['time.year'] == 1958, drop=True)['time.month'].to_numpy(),
            year=x.where(x['time.month'] == 1, drop=True)['time.year'].to_numpy(),
            lon=x.lon,
        ),
    )
    
    season_x_dt = detrend(
        season_x.compute(),
        dim='year',
    )
    return season_x_dt

In [7]:
ssh_mon = season(ssh_a)

### D20 from Nemo 

In [8]:
d20_a = monthly_anomalies(
    t_grid_file.d20.sel(**eq,time=tpart)
).fillna(0.).compute()

In [9]:
d20_y = detrend(d20_a.compute())

In [10]:
d20_mon = season(d20_a)

### NEMO-NLTI as Figure 2

In [11]:
slope = reg_slope(d20_y, ssh_y, dim="time")

nlti_y = (
    ssh_y - slope * d20_y
).rename("NLTI")

nlti_y=detrend(
    nlti_y
)

###  Month-NLTI

In [12]:
slope = reg_slope(d20_mon, ssh_mon, dim="year")

nlti_mon = (
    ssh_mon - slope * d20_mon
).rename("NLTI-mon")

nlti_mon=detrend(
    nlti_mon.compute(),
    dim='year',
)

####  Back to year

In [13]:
def back(x):
    res=np.zeros((12*61,481))
    for i in range(0,12*61):
        res[i,:] = x[i-(i//12)*12,:,i//12].drop_vars(['month','year'])
        
    back_x = xr.DataArray(
        data=res,
        dims=["time", "lon"],
        coords=dict(
            time=ssh_a.time,
            lon=x.lon,
        ),
    )
    
    return back_x

In [14]:
nlti_back = back(nlti_mon)

## Write outout

In [15]:
!rm -rf data_figS8.zarr

xr.Dataset(
    dict(
        nlti_y=nlti_y,
        nlti_mon=nlti_back,
    )
).to_zarr("data_figS8.zarr");